# Credit assignment through regression discontinuity design


## 1. The problem

In the following we consider a small network of spiking neurons $\mathbf{h} \in \mathbb{R}^n$ which is able to observe a cost function/reward signal, $C(x,y,W)$, for some computation that it is involved in. 

To adjust its weights to minimize this cost function a neuron must know the gradient:
$$
\beta_i \equiv \frac{\partial C}{\partial h_i}.
$$

This can be computed through backpropagation, $\beta_i^{BP} \equiv \beta_i$, however there are few known biologically plausible mechanisms for implementing backprop. Though some candidates have appeared recently. For instance the method of synthetic gradients uses reinforcement-learning style estimators for the gradient, $\hat{\beta}_i^{SG} \approx \beta_i^{BP}$, which addresses some of the issues with backprop.

Alternatively, in cases where the cost function is observed by the neuron, it could learn $\beta_i$ from this feedback. A na\"ive approximation of $\beta_i$ could proceed via a type of finite differences approach:
$$
\beta_i^{FD} \equiv \mathbb{E}(C|h_i = 1) - \mathbb{E}(C|h_i=0).
$$
Learning with such approaches is known to converge slowly (citations?). (also biased?)

An issue with the above estimator is that it is effectively only measuring the correlation between $h_i$ and $C$. The adage 'correlation does not imply causation' is relevant here. Note that under the hypothetical scenario that backprop has a plausible implementation mechanism, backprop provides a _causal_ measure of $\beta_i$. That is, via backprop the neuron learns directly about its effect on $C$ through all of its downstream connections. If, at some point downstream of $h_i$, all of the weights through which $h_i$ may affect $C$ are zero then backprop will correctly report $\beta_i = 0$. 

The estimator based on $\beta_i^{FD}$ on the other hand can be biased by correlations with other neurons. For instance, we can imagine in the same no-effect scenario ($\beta_i = 0$) that $\beta_i^{FD}$ could be non-zero provided there was  another neuron $h_j$ whose activity was correlated with $h_i$ and for which there were non-zero downstream weights such that $\beta_j \ne 0$. 

(This could all be written out more explicitly for a simple 2 neuron example...)

The credit assignment problem in this correlated activity case is thus a challenge: how does a neuron learn _its_ specific effect on a cost function when its neighbors are responding in a similar way to an input stimulus? Spike train correlations and their effect on coding have been well-studied (Shea-Brown studies...). (How much have correlations been studied in the context of learning?)

## 2.1 Estimation of $\beta_i$

## 2.2 Regression discontinuity design

Here we propose a modification of the FD estimator that draws on methods from causal inference, and that can recover an unbiased estimator of $\beta_i$, thus obviating the need for backprop.

(Review RDD)

The estimator thus takes the form
$$
\beta_i^{RD} \equiv \lim_{x \to \mu^+} \mathbb{E}(C|v_i = x) - \lim_{x \to \mu^-} \mathbb{E}(C|v_i=x)
$$

## 2.3 An RD learning rule

## 3.1 Model

We demonstrate this idea via a simple test system of $n=2$ leaky integrate and fire (LIF) neurons. The neurons thus obey
$$
\dot{v}_i = -g_L v_i + w_i\eta_i
$$
for $i = 1,2$. Integrate and fire means simply:
$$
v_i(t^+) = v_r, \quad \text{when }v_i(t) = v_{th}.
$$

Noisy input $\eta_i$ is comprised of a common DC current, $x$, and noise term, $\xi(t)$, plus an individual noise term, $\xi_i(t)$:
$$
\eta_i(t) = x + \sigma_i\left[\sqrt{1-c}\xi_i(t) + \sqrt{c}\xi(t)\right].
$$
The noise processes are independent white noise: $\mathbb{E}(\xi_i(t)\xi_j(t')) = \delta_{ij}\delta(t-t')$. This parameterization is chosen so that the inputs $\eta_{1,2}$ have correlation coefficient $c$. Output spike trains are defined as
$$
y_i(t) = \sum_s \delta(t-t_i^s)
$$
for $t_i^s$ the $s$th spike time.

Simulations are performed with a step size of $\Delta t = 1ms$. Thus outputs at each time step become indicator functions:
$$
h_i^t = \mathbb{1}(\text{$i$ spikes in timebin $t$}).
$$

In each timebin a cost function can then be defined as
$$
C_t(x, \mathbf{h}_t, \mathbf{w}) = (v_1 h_1^t + v_2h_2^t - x^2)^2.
$$

We simulate this system below, and show how the various estimators of $\beta_i$ relate to each other for different values of $c$ and $v_{1,2}$.

## 3.2 Simulations

In [12]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rand

dt = 0.001
t = 5
gL = 0.6
mu = 1
reset = 0
xsigma = 1
n = 2
tau = 1

In [13]:
#Initialize voltage and spike train variables
v = np.zeros((n,T))
h = np.zeros((n,T))
vt = np.zeros((n,1))
T = np.ceil(t/dt)

#Choose a random input x, and input and output weights
x = rand.randn()*xsigma
W = rand.randn(2,1)
V = rand.randn(2,1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#Simulate T seconds
for t in range(T):
    
    #Sample some noise
    
    #
    
    vt = vt + dt*dv
    
#Compute cost

TypeError: range() integer step argument expected, got float.

In [5]:
#Make some plots